In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import pickle
import numpy as np

In [2]:
from keras.models import load_model
model=load_model('chatbot_model.h5')
import json
import random
intents=json.loads(open('intents.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))

Using TensorFlow backend.
W0215 11:45:23.398027 18984 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0215 11:45:23.421360 18984 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0215 11:45:23.443923 18984 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0215 11:45:23.446923 18984 deprecation_wrapper.py:119] From C:\Users\Navya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0215 11:45:23.4569

In [3]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word.lower())for word in sentence_words]

In [4]:
#return bag of array:0 or 1 for each word in bag that exist in the sentence
def bow(sentence, words, show_details=True):
    #tokenise the pattern
    sentence_words=clean_up_sentence(sentence)
    #bag of words: matrix of N words, vocabulary matrix
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                #assign 1 if the current word is in the vocabulary position
                bag[i]=1
                if show_details:
                    print("found in bag:%s"%w)
    return(np.array(bag))

In [5]:
def predict_class(sentence,model):
    #filter out predictions below a threshold
    p=bow(sentence,words,show_details=False)
    res=model.predict(np.array([p]))[0]
    ERROR_THRESHOLD=0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    #sort by strength of the probability
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append({"intent":classes[r[0]],"probability": str(r[1])})
    return return_list

In [6]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [7]:
def chatbot_response(msg):
    ints=predict_class(msg,model)
    res=getResponse(ints,intents)
    return res

In [8]:
#creating GUI using tkinter
import tkinter
from tkinter import *

In [9]:
def send():
    msg= EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg !='':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END,"You: "+msg+'\n\n')
        ChatLog.config(foreground="#442265",font=("Verdana",12))
        
        res=chatbot_response(msg)
        ChatLog.insert(END,"Bot: "+ res +'\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [10]:
base=Tk()
base.title('Hello')
base.geometry("400x500")
base.resizable(width=FALSE,height=FALSE)


#create chat window
ChatLog=Text(base,bd=0,bg="white",height="8",width="50",font="Arial")

ChatLog.config(state=DISABLED)

In [11]:
#Bind scrollbar to chat window
scrollbar=Scrollbar(base,command=ChatLog.yview,cursor="heart")
ChatLog['yscrollcommand']=scrollbar.set

In [12]:
#create button to send the message
SendButton=Button(base,font=("Verdana",12,'bold'),text="Send", width='12',height='5',
                 bd=0,bg="#32de97",activebackground="#3c9d9b",fg='#ffffff',
                 command=send)

#create the box to enter the message
EntryBox=Text(base,bd=0,bg='white',width='29',height='5',font='Arial')

In [13]:
#Place all components on the screen
scrollbar.place(x=376,y=6,height=386)
ChatLog.place(x=6,y=6,height=386,width=370)
EntryBox.place(x=128,y=401,height=90,width=265)
SendButton.place(x=6,y=401,height=90)


base.mainloop()